In [50]:
import pandas as pd
import numpy as np

In [51]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'

# users = pd.read_csv(INPUT_PATH + 't_user.csv')
# orders = pd.read_csv(INPUT_PATH + 't_order.csv')
# loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
# loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
# clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

# loans.loan_amount = 5**loans.loan_amount
# loans_sum.loan_sum  = 5 ** loans_sum.loan_sum

In [63]:
def load_data() :

	users = pd.read_csv(INPUT_PATH + 't_user.csv')
	orders = pd.read_csv(INPUT_PATH + 't_order.csv')
	loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
	loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
	clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

	# origin price
	loans.loan_amount = 5**loans.loan_amount
	loans_sum.loan_sum  = 5 ** loans_sum.loan_sum
	orders.price = 5 ** orders.price
	orders.discount = 5 ** orders.discount 
	users.limit = 5 ** users.limit

	#loans	 
	#data to month
	loans['Date'] = pd.to_datetime(loans['loan_time'], errors='coerce')
	loans['transaction_month'] = loans['Date'].dt.month
	#fill nan with 0
	loans["loan_count"] = 1
	loans.loan_amount = loans.loan_amount.fillna(0)

	loans_new = loans.groupby(by=["uid","transaction_month"], as_index=False)["loan_amount","loan_count"].sum()
	loans_new = loans_new.pivot_table(['loan_amount',"loan_count"], ['uid'], 'transaction_month', fill_value=0)
	loans_new.reset_index( drop=False, inplace=True )
	loans_new.columns = ['{}_{}'.format(i[1], i[0]) for i in loans_new.columns]
	loans_new = loans_new.rename(index=str, columns={"_uid": "uid"})


	#orders
	orders['Date'] = pd.to_datetime(orders['buy_time'], errors='coerce')
	orders['transaction_month'] = orders['Date'].dt.month
	orders["comsume_count"] = 1

	orders["consume_amount"] = orders["price"]*orders["qty"]-orders["discount"]
	orders.consume_amount = orders.consume_amount.fillna(0)

	orders_new = orders.groupby(by=["uid","transaction_month"], as_index=False)["consume_amount","comsume_count"].sum()
	orders_new = orders_new.pivot_table(['consume_amount',"comsume_count"], ['uid'], 'transaction_month', fill_value=0)
	orders_new.reset_index( drop=False, inplace=True )
	orders_new.columns = ['{}_{}'.format(i[1], i[0]) for i in orders_new.columns]
	orders_new = orders_new.rename(index=str, columns={"_uid": "uid"})

	clicks['Date'] = pd.to_datetime(clicks['click_time'], errors='coerce')
	clicks['transaction_month'] = clicks['Date'].dt.month
	clicks["click_count"] = 1
	clicks_new = clicks.groupby(by=["uid","transaction_month"], as_index=False)["click_count"].sum()
	clicks_new = clicks_new.pivot_table('click_count', ['uid'], 'transaction_month', fill_value=0)
	clicks_new.reset_index( drop=False, inplace=True )
	clicks_new = clicks_new.rename(index=str, columns={8: "click_count_8", 9: "click_count_9", 10 : "click_count_10", 11 : "click_count_11"})

	return users, orders_new, loans_new, loans_sum, clicks_new


In [64]:
users, orders, loans, loan_sum, clicks = load_data()

In [68]:
users_orders = users.merge(orders ,how='left', on="uid")
users_orders_loans = users_orders.merge(loans ,how='left', on="uid")
user_info = users_orders_loans.merge(clicks ,how='left', on="uid")

In [70]:
list(user_info)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '9_comsume_count',
 '10_comsume_count',
 '11_comsume_count',
 '8_consume_amount',
 '9_consume_amount',
 '10_consume_amount',
 '11_consume_amount',
 '8_loan_amount',
 '9_loan_amount',
 '10_loan_amount',
 '11_loan_amount',
 '8_loan_count',
 '9_loan_count',
 '10_loan_count',
 '11_loan_count',
 'click_count_8',
 'click_count_9',
 'click_count_10',
 'click_count_11']